In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

ModuleNotFoundError: No module named 'imputer_cat'

In [5]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

# Load the train data
train_data = pd.read_csv('train.csv')

# Handle missing values
numerical_cols = train_data.select_dtypes(include=['float64']).columns
categorical_cols = train_data.select_dtypes(include=['object']).columns

imputer_num = SimpleImputer(strategy='mean')
imputer_cat = SimpleImputer(strategy='most_frequent')

# Check if columns exist before imputation
if not numerical_cols.empty:
    train_data[numerical_cols] = imputer_num.fit_transform(train_data[numerical_cols])

if not categorical_cols.empty:
    train_data[categorical_cols] = imputer_cat.fit_transform(train_data[categorical_cols])

# Label encoding for categorical columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    train_data[col] = le.fit_transform(train_data[col])
    label_encoders[col] = le

In [6]:
X = train_data.drop(['ID', 'Target'], axis=1)
y = train_data['Target']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# CatBoost Model
catboost_model = CatBoostClassifier(iterations=500, learning_rate=0.05, depth=7, verbose=100)
catboost_model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50, plot=True)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6858702	test: 0.6886560	best: 0.6886560 (0)	total: 205ms	remaining: 1m 42s
100:	learn: 0.4745677	test: 0.6119795	best: 0.6115196 (95)	total: 2.81s	remaining: 11.1s
200:	learn: 0.3707338	test: 0.5779087	best: 0.5779087 (200)	total: 4.79s	remaining: 7.13s
300:	learn: 0.2901101	test: 0.5452960	best: 0.5452960 (300)	total: 6.71s	remaining: 4.43s
400:	learn: 0.2393663	test: 0.5399701	best: 0.5392554 (377)	total: 8.81s	remaining: 2.17s
499:	learn: 0.2077148	test: 0.5337008	best: 0.5335886 (496)	total: 11s	remaining: 0us

bestTest = 0.5335886251
bestIteration = 496

Shrink model to first 497 iterations.


In [7]:
y_val_pred_catboost = catboost_model.predict(X_val)
f1_catboost = f1_score(y_val, y_val_pred_catboost)
print(f"F1 Score CatBoost: {f1_catboost}")

# Neural Network Model
model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Convert target to binary
y_train_nn = y_train.astype(float)
y_val_nn = y_val.astype(float)


F1 Score CatBoost: 0.7760736196319018


c:\Users\SARVESH\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.fit(X_train, y_train_nn, epochs=50, batch_size=32, validation_data=(X_val, y_val_nn), verbose=1)


Epoch 1/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - accuracy: 0.5015 - loss: 5.1619 - val_accuracy: 0.5789 - val_loss: 0.7749
Epoch 2/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5653 - loss: 0.7525 - val_accuracy: 0.5620 - val_loss: 0.9487
Epoch 3/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5694 - loss: 0.7600 - val_accuracy: 0.4718 - val_loss: 0.9459
Epoch 4/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5372 - loss: 0.9237 - val_accuracy: 0.5846 - val_loss: 0.7043
Epoch 5/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5711 - loss: 0.7669 - val_accuracy: 0.5545 - val_loss: 0.8988
Epoch 6/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5854 - loss: 0.7656 - val_accuracy: 0.6053 - val_loss: 0.7208
Epoch 7/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5665 - loss: 0.8023 - val_accuracy: 0.6128 - val_loss: 0.6663
Epoch 8/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5667 - loss: 0.7623 - val_accuracy: 0.5883 - val_loss

In [10]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

# Load the test data
test_data = pd.read_csv('test.csv')

# Handle missing values
numerical_cols = test_data.select_dtypes(include=['float64']).columns
categorical_cols = test_data.select_dtypes(include=['object']).columns

imputer_num = SimpleImputer(strategy='mean')
imputer_cat = SimpleImputer(strategy='most_frequent')

# Check if columns exist before imputation
if not numerical_cols.empty:
    imputer_num.fit(test_data[numerical_cols])
    test_data[numerical_cols] = imputer_num.transform(test_data[numerical_cols])

if not categorical_cols.empty:
    imputer_cat.fit(test_data[categorical_cols])
    test_data[categorical_cols] = imputer_cat.transform(test_data[categorical_cols])

# Label encoding for categorical columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    test_data[col] = le.fit_transform(test_data[col])
    label_encoders[col] = le

# Predict using CatBoost for submission
test_predictions_catboost = catboost_model.predict(test_data.drop('ID', axis=1))

# Create submission file for CatBoost
submission_catboost = pd.DataFrame({'ID': test_data['ID'], 'Target': test_predictions_catboost})
submission_catboost.to_csv('submission_catboost.csv', index=False)
print("CatBoost submission file saved successfully!")

# Predict using Neural Network for submission
test_predictions_nn = (model.predict(test_data.drop('ID', axis=1)) > 0.5).astype(int).flatten()

# Create submission file for Neural Network
submission_nn = pd.DataFrame({'ID': test_data['ID'], 'Target': test_predictions_nn})
submission_nn.to_csv('submission_nn.csv', index=False)
print("Neural Network submission file saved successfully!")


CatBoost submission file saved successfully!
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Neural Network submission file saved successfully!
